In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "ReadFromHDFS")

22/05/05 21:41:25 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.174.129 instead (on interface ens33)
22/05/05 21:41:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/05 21:41:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/05 21:41:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/05 21:41:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/05 21:41:27 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [7]:
# read a text file from HDFS, movies.csv
# hdfs dfs -ls /movies
rdd = sc.textFile("hdfs://localhost:9000/ml-latest-small/movies.csv")

In [9]:
# get number of lin
result = rdd.count()
print(result)

9743


In [10]:
rdd.first()

22/05/05 21:54:26 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:531)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3441)
	at org.apache.hadoop.hdfs.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:777)
	at org.apache.hadoop.hdfs.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:694)
	at org.apache.hadoop.hdfs.BlockReaderFactory.build(BlockReaderFactory.java:355)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:665)
	at org.apache.hadoop.hdfs.DFSInputStream.seekToBlockSource(DFSInputStream.java:1567)
	at org.apache.hadoop.hdfs.DFSInputStream.re

'movieId,title,genres'

In [11]:
header = rdd.first()

In [12]:
print  ( rdd.take(5) )


['movieId,title,genres', '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy', '2,Jumanji (1995),Adventure|Children|Fantasy', '3,Grumpier Old Men (1995),Comedy|Romance', '4,Waiting to Exhale (1995),Comedy|Drama|Romance']


In [13]:
# to skip first line, 
headerLine = rdd.first()
# no header data in rddContent
rddContent = rdd.filter (lambda line: line != headerLine)

print (rddContent.take(5) )

['1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy', '2,Jumanji (1995),Adventure|Children|Fantasy', '3,Grumpier Old Men (1995),Comedy|Romance', '4,Waiting to Exhale (1995),Comedy|Drama|Romance', '5,Father of the Bride Part II (1995),Comedy']


In [14]:
# then each split line into list 
line = '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy'
out = line.split(",") # assignment

print(out[0]) # ,movie id
print(out[1]) # title
print(out[2]) # genres

out # expression

1
Toy Story (1995)
Adventure|Animation|Children|Comedy|Fantasy


['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy']

In [15]:
# then each split line into list 
line = '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy'
out = line.split(",") # assignment

print(out[0]) # ,movie id
print(out[1]) # title
print(out[2]) # genres

out # expression

1
Toy Story (1995)
Adventure|Animation|Children|Comedy|Fantasy


['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy']

In [16]:
# moviesRdd is list of list consist of elements parsed using ,
# split returns list, we can convert into tuple
moviesRdd = rddContent.map (lambda line: tuple(line.split(",")))

print(moviesRdd.take(5))

[('1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'), ('2', 'Jumanji (1995)', 'Adventure|Children|Fantasy'), ('3', 'Grumpier Old Men (1995)', 'Comedy|Romance'), ('4', 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance'), ('5', 'Father of the Bride Part II (1995)', 'Comedy')]


In [21]:
moviesRdd2 = moviesRdd.map(lambda t: (int(t[0]), t[1].upper(), t[2].lower()))
print(moviesRdd2.take(5))

[(1, 'TOY STORY (1995)', 'adventure|animation|children|comedy|fantasy'), (2, 'JUMANJI (1995)', 'adventure|children|fantasy'), (3, 'GRUMPIER OLD MEN (1995)', 'comedy|romance'), (4, 'WAITING TO EXHALE (1995)', 'comedy|drama|romance'), (5, 'FATHER OF THE BRIDE PART II (1995)', 'comedy')]


In [33]:
#use moviesRdd2 and filter , print the movies between 100 and 105
filterRdd = moviesRdd2.filter(lambda t: t[0] >= 100 and t[0] <=105)
print(filterRdd.take(6))

[(100, 'CITY HALL (1996)', 'drama|thriller'), (101, 'BOTTLE ROCKET (1996)', 'adventure|comedy|crime|romance'), (102, 'MR. WRONG (1996)', 'comedy'), (103, 'UNFORGETTABLE (1996)', 'mystery|sci-fi|thriller'), (104, 'HAPPY GILMORE (1996)', 'comedy'), (105, '"BRIDGES OF MADISON COUNTY', ' the (1995)"')]


In [34]:
s = 'adventure|comedy|crime|romance'
print('comedy' in s )
print('mystery' in s)


True
False


In [37]:
filterS = moviesRdd2.filter(lambda t: 'comedy' in t[2])
print(filterS.take(10))

[(1, 'TOY STORY (1995)', 'adventure|animation|children|comedy|fantasy'), (3, 'GRUMPIER OLD MEN (1995)', 'comedy|romance'), (4, 'WAITING TO EXHALE (1995)', 'comedy|drama|romance'), (5, 'FATHER OF THE BRIDE PART II (1995)', 'comedy'), (7, 'SABRINA (1995)', 'comedy|romance'), (12, 'DRACULA: DEAD AND LOVING IT (1995)', 'comedy|horror'), (18, 'FOUR ROOMS (1995)', 'comedy'), (19, 'ACE VENTURA: WHEN NATURE CALLS (1995)', 'comedy'), (20, 'MONEY TRAIN (1995)', 'action|comedy|crime|drama|thriller'), (21, 'GET SHORTY (1995)', 'comedy|crime|thriller')]


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 44870)
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/ubuntu/miniconda3/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/ubuntu/miniconda3/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/ubuntu/miniconda3/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/opt/spark-3.1.3-bin-hadoop2.7/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/opt/spark-3.1.3-bin-hadoop2.7/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/opt/spark-3.1.3-bin-hadoop2.7/python/pyspark/accumulators.py", line 239